LangSmith is a platform built specifically for debugging, testing, and monitoring LLM applications. For LangGraph, it's an indispensable tool that gives you an X-ray view into your graph's execution, turning a complex black box into a transparent, explorable flowchart.

**What LangSmith Provides**<br>
Instead of trying to follow a messy stream of print statements, LangSmith gives you a rich, interactive UI where you can see:

A full trace of your graph's execution path, node by node.

The exact inputs (the state) each node received.

The exact outputs (the state updates) each node produced.

The full prompt and response for every LLM call.

The inputs and outputs for every tool call.

A "diff" view showing precisely how the state changed at each step.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Hierarchical Agent Teams"

In [2]:
import os
import json
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults



tool = TavilySearchResults(max_results=3)
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, gemini_api_key=os.getenv("GOOGLE_API_KEY"))

# ==============================================================================
# PART 1: CREATE THE WORKER GRAPH (Our Adaptive RAG Agent)
# This is a self-contained agent that can research a topic and self-correct.
# ==============================================================================

class AdaptiveRAGState(TypedDict):
    """The state for our worker agent."""
    messages: Annotated[List[BaseMessage], add_messages]

def create_adaptive_rag_graph():
    """Factory function to create the worker graph."""
    
    # Define worker nodes
    def retrieval_node(state: AdaptiveRAGState):
        print("---WORKER: RETRIEVAL---")
        query = state['messages'][-1].content
        retrieved_docs = tool.invoke({"query": query})
        doc_text = "\n\n".join(str(d) for d in retrieved_docs)
        return {"messages": [HumanMessage(content=doc_text, name="Retriever")]}

    def assessment_node(state: AdaptiveRAGState):
        print("---WORKER: ASSESSMENT---")
        system_prompt = """You are a relevance assessor. Your task is to evaluate if the retrieved documents are sufficient to answer the user's question.
        Respond with a JSON object with one key, 'is_relevant': a boolean."""
        retrieved_docs = state['messages'][-1].content
        user_question = state['messages'][-2].content
        prompt = f"User Question: {user_question}\n\nRetrieved Documents:\n{retrieved_docs}"
        response = model.invoke([SystemMessage(content=system_prompt), HumanMessage(content=prompt)])
        return {"messages": [HumanMessage(content=response.content, name="Assessor")]}

    def generation_node(state: AdaptiveRAGState):
        print("---WORKER: GENERATION---")
        prompt = f"Based on the following documents, please provide a comprehensive answer to this question:\n\nQuestion: {state['messages'][0].content}\n\nDocuments:\n{state['messages'][-2].content}"
        response = model.invoke(prompt)
        return {"messages": [response]}

    # Define worker router
    def relevance_router(state: AdaptiveRAGState) -> str:
        print("---WORKER ROUTER---")
        assessment_message = state['messages'][-1].content
        try:
            assessment_json = json.loads(assessment_message)
            if assessment_json.get('is_relevant'):
                return "generate"
            else:
                return "retrieve" # Loop back to retrieve with the same query for simplicity
        except (json.JSONDecodeError, KeyError):
            return "end"

    # Build the worker graph
    builder = StateGraph(AdaptiveRAGState)
    builder.add_node("retriever", retrieval_node)
    builder.add_node("assessor", assessment_node)
    builder.add_node("generator", generation_node)
    
    builder.set_entry_point("retriever")
    builder.add_edge("retriever", "assessor")
    builder.add_conditional_edges("assessor", relevance_router, {
        "generate": "generator",
        "retrieve": "retriever",
        "end": END,
    })
    builder.add_edge("generator", END)
    
    return builder.compile()

# ==============================================================================
# PART 2: CREATE THE MANAGER GRAPH
# This graph manages a list of tasks and delegates each one to the worker.
# ==============================================================================

# First, create an instance of our worker graph.
research_worker_graph = create_adaptive_rag_graph()

# Define the Manager's state
class ManagerState(TypedDict):
    topics_to_research: List[str]
    current_topic: str
    results: Annotated[List[str], lambda x, y: x + y] # Reducer to append results

# Define the Manager's nodes
def select_topic_node(state: ManagerState):
    """Selects the next topic from the list to be researched."""
    print(f"---MANAGER: {len(state['topics_to_research'])} topics left---")
    topic = state['topics_to_research'][0]
    remaining_topics = state['topics_to_research'][1:]
    return {"current_topic": topic, "topics_to_research": remaining_topics}

def researcher_proxy_node(state: ManagerState):
    """This is the key node. It invokes the worker graph."""
    topic = state['current_topic']
    print(f"---MANAGER: DELEGATING '{topic}' TO WORKER---")
    
    # Invoke the worker graph with the current topic
    worker_response = research_worker_graph.invoke(
        {"messages": [HumanMessage(content=topic)]}
    )
    
    # The worker's final answer is in its last message.
    final_answer = worker_response['messages'][-1].content
    
    # Append the result to the manager's list of results
    return {"results": [f"Topic: {topic}\n\n{final_answer}"]}

# Define the Manager's router
def manager_router(state: ManagerState):
    """Routes to the end if there are no more topics to research."""
    if not state['topics_to_research']:
        return "end"
    else:
        return "continue"

# Build the Manager graph
manager_builder = StateGraph(ManagerState)
manager_builder.add_node("select_topic", select_topic_node)
manager_builder.add_node("research_worker", researcher_proxy_node)

manager_builder.set_entry_point("select_topic")
manager_builder.add_edge("select_topic", "research_worker")
manager_builder.add_conditional_edges("research_worker", manager_router, {
    "continue": "select_topic",
    "end": END
})

manager_graph = manager_builder.compile()

# ==============================================================================
# PART 3: EXECUTE THE HIERARCHY
# ==============================================================================

# The list of topics for the manager to delegate.
topics = ["The future of AI hardware", "Recent advancements in large language models"]
initial_state = {"topics_to_research": topics, "results": []}

# Stream the results from the manager graph.
print("\n---EXECUTING HIERARCHICAL TEAM---")
final_result = manager_graph.invoke(initial_state)

print("\n\n" + "="*80)
print("--- HIERARCHY EXECUTION COMPLETE ---")
print("Collected results:")
for i, result in enumerate(final_result['results']):
    print(f"\n--- RESULT {i+1} ---")
    print(result)

C:\Users\ujwal\AppData\Local\Temp\ipykernel_19996\3929291860.py:12: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=3)
Unexpected argument 'gemini_api_key' provided to ChatGoogleGenerativeAI. Did you mean: 'google_api_key'?
c:\Users\ujwal\OneDrive\Documents\GitHub\LangGraph\venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: UserWarning: WARNING! gemini_api_key is not default parameter.
                gemini_api_key was transferred to model_kwargs.
                Please confirm that gemini_api_key is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)



---EXECUTING HIERARCHICAL TEAM---
---MANAGER: 2 topics left---
---MANAGER: DELEGATING 'The future of AI hardware' TO WORKER---
---WORKER: RETRIEVAL---
---WORKER: ASSESSMENT---
---WORKER ROUTER---
---MANAGER: 1 topics left---
---MANAGER: DELEGATING 'Recent advancements in large language models' TO WORKER---
---WORKER: RETRIEVAL---
---WORKER: ASSESSMENT---
---WORKER ROUTER---


--- HIERARCHY EXECUTION COMPLETE ---
Collected results:

--- RESULT 1 ---
Topic: The future of AI hardware

```json
{
  'is_relevant': True
}
```

--- RESULT 2 ---
Topic: Recent advancements in large language models

```json
{
  "is_relevant": true
}
```


### Streaming and Asynchronous Operations
 you use yield to turn a regular function into a generator that can stream out multiple values over time instead of just returning one value at the end.

In [31]:
import asyncio
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END
from operator import add
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI

# --- 1. State and Model ---
class StreamState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    # This will hold the streamed, final poem
    poem: Annotated[str, lambda old, new: new]  # REPLACE, not add

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)

# --- 2. The Streaming Node ---
# This is the most important change.
async def writer_node(state: StreamState):
    """A node that streams the output of an LLM."""
    print("---STREAMING WRITER---")
    # Get the topic from the last message
    prompt = f"Write a short, four-line poem about {state['messages'][-1].content}."
    
    # Use model.stream() to get a stream of tokens
    stream = model.astream(prompt)
    
    # Yield a dictionary for each chunk from the stream
    async for chunk in stream:
        # Each chunk is a piece of the poem. We yield an update to the 'poem' key.
        yield {**state,"poem": chunk.content}

# --- 3. Build The Graph ---
# A very simple graph: just the writer, which then ends.
builder = StateGraph(StreamState)
builder.set_entry_point("writer")
builder.add_node("writer", writer_node)
builder.add_edge("writer", END)
graph = builder.compile()

# --- 4. Consume the Stream with .astream_log() ---
async def run_agent():
    """Runs the agent and prints the streamed output in real-time."""
    # We use .astream_log() to get a detailed stream of events.
    full_poem = ""  # This will track the full poem as it builds
    async for op in graph.astream_log(
        {"messages": [HumanMessage(content="the moon")],"poem":""},
        
        config={"include_values":True} # This includes the full state view in the log
    ):
        # The log contains many operations. We're interested in the ones
        # that update our 'poem' state.
        for op in op.ops:
            # We're interested in operations that update our 'poem' state.
            # The path for this is '/values/poem'.
            if op["op"] == "replace" and op["path"] == "/values/poem":
                # op['value'] contains the full poem after this update.
                # We diff it with the previous version to get the new chunk.
                new_chunk = op["value"][len(full_poem):]
                print(new_chunk, end="", flush=True)
                full_poem = op["value"] # Update our tracker
    print("\n---STREAMING COMPLETE---")

# To run an async function in a notebook or script:
# In a Jupyter Notebook, you can just `await run_agent()`.
# In a standard .py file, you run it like this:
"""if __name__ == "__main__":
    asyncio.run(run_agent())"""
await run_agent()  

---STREAMING WRITER---

---STREAMING COMPLETE---


IDK why all the code in world is not making it stream

In [32]:
await run_agent()

---STREAMING WRITER---

---STREAMING COMPLETE---


In [16]:
def generator_function():
    print("Sending first chunk")
    yield "First" # Pauses here
    
    print("Resuming and sending second chunk")
    yield "Second" # Pauses here
    
    print("Resuming and sending final chunk")
    yield "Third" # Pauses and ends

# How you use it:
for chunk in generator_function():
    print(f"Received: {chunk}")

# Output:
# Sending first chunk
# Received: First
# Resuming and sending second chunk
# Received: Second
# Resuming and sending final chunk
# Received: Third

Sending first chunk
Received: First
Resuming and sending second chunk
Received: Second
Resuming and sending final chunk
Received: Third
